# Thai N-NER: Thai Nested Named Entity Recognition

This demo notebook provides a tutorial on using Thai N-NER, with references from [Thai N-NER](https://medium.com/airesearch-in-th/thai-n-ner-thai-nested-named-entity-recognition-1969f8fe91f0)

Learn more about Thai N-NER here : [Thai N-NER](https://medium.com/airesearch-in-th/thai-n-ner-thai-nested-named-entity-recognition-1969f8fe91f0)

## 1. Setup and Preprocessing

In [13]:
# !pip install seqeval
# !pip install pythainlp
# !pip install transformers==4.29.2
# !pip install sentencepiece
# !pip install gdown
# !pip install thai_nner
# !pip install protobuf==3.20.3

# Model checkpoints

> Thai N-NER provides necessary resources, including models, datasets, and pre-trained language models, available here : [Thai N-NER (resources)](https://drive.google.com/drive/folders/1Dy-360iZ9hIA-xA0yizSwmpM8sx6rrjJ?usp=sharing)

To utilize this, please follow these steps::

1. Add the Shared Folder [Thai N-NER (resources)](https://drive.google.com/drive/folders/1Dy-360iZ9hIA-xA0yizSwmpM8sx6rrjJ?usp=sharing)  to Your Google Drive.
* first open the shared folder link in your web browser
* Click the folder named "thai-nner" at the top of the page.
* In the menu bar, click "Organize", then click "Add shortcut" to Drive (you may see an icon that looks like a Drive logo with a plus sign)
* Select "My Drive"


In [14]:
# Clone github
# !git clone https://github.com/vistec-AI/Thai-NNER.git
%cd ./Thai-NNER

[Errno 2] No such file or directory: './Thai-NNER'
/home/andre/Desktop/CU_submission/NLP_2025/L04_Token_Classification/Demo_NER/Thai-NNER


# Mount your drive to Google Colab.

In [15]:
# Load data
# from google.colab import drive
# drive.mount('/content/drive/')

# # Create symbolic links
# !ln -s "./thai-nner/lm" ./data/lm
# !ln -s "./thai-nner/checkpoints" ./data/checkpoints

# Inference

In [16]:
import json
import torch
import argparse
from tqdm import tqdm
from tabulate import tabulate

from utils.unique import unique
import model.loss as module_loss
import model.model as module_arch
import model.metric as module_metric
from parse_config import ConfigParser
import data_loader.data_loaders as module_data

PAD = '<pad>'

In [17]:
resume = 'data/checkpoints/1102_151935/checkpoint.pth'

In [19]:
args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default=None, type=str, help='config file path (default: None)')
args.add_argument('-r', '--resume', default=f"{resume}", type=str, help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default=None, type=str, help='indices of GPUs to enable (default: all)')
args.add_argument('-f', '--file', default=None, type=str, help='Error')
config = ConfigParser.from_args(args)
logger = config.get_logger('test')

# build model architecturea
model = config.init_obj('arch', module_arch)

# get function handles of loss and metrics
criterion = getattr(module_loss, config['loss'])
metric_fns = [getattr(module_metric, met) for met in config['metrics']]

logger.info('Loading checkpoint: {} ...'.format(config.resume))
checkpoint = torch.load(config.resume)
state_dict = checkpoint['state_dict']

if config['n_gpu'] > 1:
    model = torch.nn.DataParallel(model)
print(config['n_gpu'])

model.load_state_dict(state_dict, strict=False)
layers_train = config._config['trainer']['layers_train']

# prepare model for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

total_loss = 0.0
total_metrics = torch.zeros(len(metric_fns))
# logger.info(model)

Loading checkpoint: data/checkpoints/1102_151935/checkpoint.pth ...


/tmp/ipykernel_11305/2011583196.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(config.resume)


4


In [20]:
# Loading only few testing examples.
config.config['data_loader']['args']['sample_data'] = True

data_loader = config.init_obj('data_loader', module_data)
test_data_loader = data_loader.get_test()

> Now, Let's try using the pre-trained Thai N-NER model checkpoint to perform inference and predict NE tags.

In [21]:
from utils.prediction import predict, get_dict_prediction, show



text = " วันนี้วันที่ 27 มกราคม 2568 เป็นวันที่อากาศดีมาก "


tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];

<s>|วันนี้|วันที่|27|มกราคม|25|68|เป็น|วันที่||อากาศ||ดีมาก|</s> 

[1, 2]         rel            วันนี้
[2, 4]         day            วันที่27
[2, 7]         date           วันที่27มกราคม2568
[3, 4]         cardinal       27
[4, 5]         month          มกราคม
[5, 7]         year           2568


In [22]:
text = "คณะกรรมการ 40 ปี 14 ตุลาเพื่อประชาธิปไตรสมบูรณ์"
tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];

<s>||คณะกรรมการ|40|ปี|14||ตุ|ลา|เพื่อ||ประชา||ธิ|ป||ไต|ร||สมบูรณ์|</s> 

[3, 4]         cardinal       40
[3, 4]         cardinal       40
[3, 5]         duration       40ปี
[4, 5]         unit           ปี
[5, 6]         day            14
[6, 9]         month          ตุลา
[10, 20]       norp_political ประชาธิปไตรสมบูรณ์


In [23]:
text = " วันที่ 18 มกราคม 2568 เมื่อเวลา 11.15 น. ที่จ.นครพนม นายทักษิณ ชินวัตร อดีตนายกฯ ให้สัมภาษณ์กรณีนายชาดา ไทยเศรษฐ์ อดีต รมช.มหาดไทย เซ็นคำสั่งเพิกถอนที่ดินสนามกอล์ฟอัลไพน์ กลับคืนเป็นที่ธรณีสงฆ์ ก่อนหมดวาระเพียงไม่กี่วัน "
tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];

<s>|วันที่|18|มกราคม|25|68|เมื่อ|เวลา|11.|15|น|.|ที่|จ|.||นคร|พ|นม|นาย||ทักษิณ||ชิน|วั|ตร||อดีต|นาย|ก|ฯ|ให้||สัมภาษณ์||กรณี|นาย||ชา||ดา||ไทย||เศรษฐ|์||อดีต||รม|ช|.|ม|หาด|ไทย||เซ็น||คําสั่ง||เพ|ิก|ถอน||ที่ดิน||สนาม|กอล์ฟ||อัล|ไพ|น์||กลับ|คืน|เป็น|ที่|ธร|ณี|สงฆ์|ก่อน||หมด|วา|ระ||เพียง|ไม่||กี่|วัน|</s> 

[1, 3]         day            วันที่18
[1, 6]         date           วันที่18มกราคม2568
[2, 3]         cardinal       18
[3, 4]         month          มกราคม
[4, 6]         year           2568
[7, 12]        time           เวลา11.15น.
[8, 10]        cardinal       11.15
[8, 12]        time           11.15น.
[10, 12]       unit           น.
[13, 19]       province       จ.นครพนม
[15, 19]       province       นครพนม
[19, 20]       title          นาย
[19, 26]       person         นายทักษิณชินวัตร
[20, 22]       firstname      ทักษิณ
[22, 26]       last           ชินวัตร
[26, 31]       role           อดีตนายกฯ
[36, 37]       title          นาย
[36, 46]       person         นายชาดาไทยเศรษฐ์
[

In [24]:
text = " สธ.กางตัวเลขเบื้องต้นคนป่วยจากปัญหาฝุ่น PM2.5 แค่ 3 สัปดาห์ของเดือน ม.ค.พุ่ง 144,000 คนส่วนใหญ่ผิวหนัง ตาอักเสบ โรคหืด พบ 5 จังหวัดค่าฝุ่นเกิน 75 มคก.ต่อ ลบ.ม.ต่อเนื่องเกิน 3 ในระดับสีแดง "
tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];

<s>|ส|ธ||.||กา|ง||ตัวเลข||เบื้องต้น|คน|ป่วย|จาก||ปัญหา||ฝุ่น|PM|2.5||แค่|3||สัปดาห์|ของ|เดือน|ม|.|ค|.||พุ่ง|14|4,000|คน||ส่วนใหญ่||ผิวหนัง||ตา||อักเสบ||โรค|ห|ืด||พบ|5|จังหวัด|ค่า||ฝุ่น||เกิน|75|ม|ค|ก|.|ต่อ||ลบ||.|ม|.||ต่อเนื่อง||เกิน|3|ใน||ระดับ||สีแดง|</s> 

[1, 5]         goverment      สธ.
[23, 24]       cardinal       3
[23, 26]       duration       3สัปดาห์
[24, 26]       unit           สัปดาห์
[27, 32]       month          เดือนม.ค.
[28, 32]       month          ม.ค.
[34, 36]       cardinal       144,000
[34, 37]       quantity       144,000คน
[36, 37]       unit           คน
[41, 45]       disease        ตาอักเสบ
[51, 52]       cardinal       5
[52, 53]       unit           จังหวัด
[58, 59]       cardinal       75
[59, 63]       unit           มคก.
[64, 70]       unit           ลบ.ม.
[74, 75]       cardinal       3
